# 📋 Task 2: Domain Generalization via Invariant & Robust Learning

In this task, we explore Domain Generalization (DG), where a model is trained on multiple source domains and must generalize to a completely unseen target domain. We will implement and compare four methods: ERM, IRM, GroupDRO, and SAM.

Our setup will use the **PACS dataset**. We will train on the **Art, Cartoon, and Photo** domains, holding out the **Sketch** domain as our unseen test environment, as suggested in the assignment manual.

---

## **Part 1: Empirical Risk Minimization (ERM) Baseline**

### **1.1. Overview**

We begin by establishing a baseline using standard **Empirical Risk Minimization (ERM)**. This approach involves merging all data from the source domains into a single dataset and training a standard classifier on it. This model's performance on the unseen target domain will serve as the benchmark against which we will compare more advanced DG techniques.

### **1.2. Environment Setup**

First, we need to set up the Python environment to ensure the notebook can find and import the DomainBed library from our `code/` directory.

In [1]:
import json
import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# The path to the 'domainbed' repository inside the 'code' folder
# This is the parent directory of the actual 'domainbed' package
module_path = os.path.abspath(os.path.join(".", "code", "domainbed"))

if module_path not in sys.path:
    sys.path.append(module_path)
    print(f"✅ Added '{module_path}' to Python path.")

# Import the main training function from DomainBed
try:
    from domainbed.scripts import train

    print("✅ Successfully imported DomainBed.")
except ImportError as e:
    print(
        "❌ Error importing DomainBed. Check that the path is correct and the repository is at './code/domainbed'."
    )
    print(e)

# Set plotting style for later
sns.set_theme(style="whitegrid")

✅ Added '/root/IbsATML/PA2/Domain Generalisation/code/domainbed' to Python path.


/venv/main/lib/python3.12/site-packages/outdated/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


✅ Successfully imported DomainBed.


### **1.3. Experiment Runner Function**

To keep our code clean, we'll define a helper function that can launch any DomainBed experiment by taking a dictionary of arguments. This function mimics passing arguments via the command line.

In [2]:
def run_experiment(args_dict):
    """
    Builds a command-line command from a dictionary of arguments
    and executes the DomainBed training script, setting the PYTHONPATH.
    """
    # Define the path to the directory containing the 'domainbed' package
    module_path = os.path.abspath(os.path.join('.', 'code', 'domainbed'))
    
    # FIX 1: Enclose the module_path in quotes to handle spaces in the directory name.
    command = f'PYTHONPATH="{module_path}" python -m domainbed.scripts.train'
    
    # Append each argument from the dictionary to the command string
    for key, value in args_dict.items():
        if isinstance(value, bool) and value:
            command += f" --{key}"
        elif not (isinstance(value, bool) and not value):
            command += f" --{key} {value}"
            
    print("🚀 Executing Command:")
    print(command)
    
    # Execute the command in the shell
    os.system(command)
    
    # FIX 2: Corrected the typo 'N'/'A' to the string 'N/A'.
    print(f"\n🎉 Training finished for {args_dict.get('algorithm', 'N/A')}.")

### **1.4. Run ERM Training**

Now, we define the specific parameters for our ERM baseline experiment and launch the training.

In [3]:
# --- ERM Experiment Configuration ---

# 1. Define hyperparameters in their own dictionary.
hparams = {
    'progress_bar': True
    # You can add other hyperparameters like batch_size or lr here too.
    # 'batch_size': 32,
    # 'lr': 5e-5
}

# 2. Configure the main experiment arguments.
erm_args = {
    'data_dir': './data/',
    'dataset': 'PACS',
    'algorithm': 'ERM',
    'test_env': 3,  # The index for the 'Sketch' domain in PACS
    'output_dir': './results/erm',
    'hparams_seed': 0,
    'trial_seed': 0,
    'seed': 0,
    # 3. Add the hparams dictionary as a JSON string.
    # The command line expects a string, so we use json.dumps().
    # We wrap it in single quotes for the shell.
    'hparams': f"'{json.dumps(hparams)}'"
}

# Launch the experiment
run_experiment(erm_args)

🚀 Executing Command:
PYTHONPATH="/root/IbsATML/PA2/Domain Generalisation/code/domainbed" python -m domainbed.scripts.train --data_dir ./data/ --dataset PACS --algorithm ERM --test_env 3 --output_dir ./results/erm --hparams_seed 0 --trial_seed 0 --seed 0 --hparams '{"progress_bar": true}'


/venv/main/lib/python3.12/site-packages/outdated/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Environment:
	Python: 3.12.11
	PyTorch: 2.8.0+cu129
	Torchvision: 0.23.0+cu129
	CUDA: 12.9
	CUDNN: 91002
	NumPy: 2.1.2
	PIL: 11.0.0
Args:
	algorithm: ERM
	checkpoint_freq: None
	data_dir: ./data/
	dataset: PACS
	holdout_fraction: 0.2
	hparams: {"progress_bar": true}
	hparams_seed: 0
	output_dir: ./results/erm
	save_model_every_checkpoint: False
	seed: 0
	skip_model_save: False
	steps: None
	task: domain_generalization
	test_envs: [3]
	trial_seed: 0
	uda_holdout_fraction: 0
HParams:
	batch_size: 32
	class_balanced: False
	data_augmentation: True
	dinov2: False
	freeze_bn: False
	lars: False
	linear_steps: 500
	lr: 5e-05
	nonlinear_classifier: False
	progress_bar: True
	resnet18: False
	resnet50_augmix: True
	resnet_dropout: 0.0
	vit: False
	vit_attn_tune: False
	vit_dropout: 0.0
	weight_decay: 0.0
/venv/main/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights

---

## **Part 2: Invariant Risk Minimization (IRM)**

### **2.1. Overview**

Now we move to **Invariant Risk Minimization (IRM)**, as required by Part 2 of the assignment. The core idea behind IRM is to learn a feature representation where the optimal classifier is the same across all training domains. This is intended to prevent the model from relying on spurious, domain-specific correlations, thereby improving generalization to unseen domains.

We will first run IRM with its default hyperparameters from DomainBed and then perform an ablation study with a stronger penalty weight to analyze its stability and performance.

### **2.2. Run IRM Training (Default Hyperparameters)**

We start with the default IRM penalty `irm_lambda=1e2`.

In [4]:
# --- IRM Experiment Configuration (Default) ---

# 1. Define hyperparameters for the default IRM run.
hparams_irm_default = {
    'progress_bar': True,
    'irm_lambda': 1e2,               # Default penalty weight
    'irm_penalty_anneal_iters': 500  # Steps to anneal the penalty
}

# 2. Configure the main experiment arguments.
irm_args_default = {
    'data_dir': './data/',
    'dataset': 'PACS',
    'algorithm': 'IRM',
    'test_env': 3,  # Sketch domain
    'output_dir': './results/irm_default',
    'hparams_seed': 0,
    'trial_seed': 0,
    'seed': 0,
    # 3. Add the hparams dictionary as a JSON string.
    'hparams': f"'{json.dumps(hparams_irm_default)}'"
}

# Launch the experiment
run_experiment(irm_args_default)

🚀 Executing Command:
PYTHONPATH="/root/IbsATML/PA2/Domain Generalisation/code/domainbed" python -m domainbed.scripts.train --data_dir ./data/ --dataset PACS --algorithm IRM --test_env 3 --output_dir ./results/irm_default --hparams_seed 0 --trial_seed 0 --seed 0 --hparams '{"progress_bar": true, "irm_lambda": 100.0, "irm_penalty_anneal_iters": 500}'


/venv/main/lib/python3.12/site-packages/outdated/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Environment:
	Python: 3.12.11
	PyTorch: 2.8.0+cu129
	Torchvision: 0.23.0+cu129
	CUDA: 12.9
	CUDNN: 91002
	NumPy: 2.1.2
	PIL: 11.0.0
Args:
	algorithm: IRM
	checkpoint_freq: None
	data_dir: ./data/
	dataset: PACS
	holdout_fraction: 0.2
	hparams: {"progress_bar": true, "irm_lambda": 100.0, "irm_penalty_anneal_iters": 500}
	hparams_seed: 0
	output_dir: ./results/irm_default
	save_model_every_checkpoint: False
	seed: 0
	skip_model_save: False
	steps: None
	task: domain_generalization
	test_envs: [3]
	trial_seed: 0
	uda_holdout_fraction: 0
HParams:
	batch_size: 32
	class_balanced: False
	data_augmentation: True
	dinov2: False
	freeze_bn: False
	irm_lambda: 100.0
	irm_penalty_anneal_iters: 500
	lars: False
	linear_steps: 500
	lr: 5e-05
	nonlinear_classifier: False
	progress_bar: True
	resnet18: False
	resnet50_augmix: True
	resnet_dropout: 0.0
	vit: False
	vit_attn_tune: False
	vit_dropout: 0.0
	weight_decay: 0.0
/venv/main/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWa

### **2.3. Ablation Study: Run IRM with Stronger Penalty**

As suggested in the assignment, we will now conduct a stability analysis by increasing the penalty weight by an order of magnitude to `irm_lambda=1e3`. This will help us understand how sensitive the IRM algorithm is to this hyperparameter.

In [5]:
# --- IRM Experiment Configuration (Stronger Penalty) ---

# 1. Define hyperparameters with the increased penalty.
hparams_irm_stronger = {
    'progress_bar': True,
    'irm_lambda': 1e3,               # 10x stronger penalty
    'irm_penalty_anneal_iters': 500
}

# 2. Configure the main experiment arguments.
irm_args_stronger = {
    'data_dir': './data/',
    'dataset': 'PACS',
    'algorithm': 'IRM',
    'test_env': 3,  # Sketch domain
    'output_dir': './results/irm_stronger_penalty',
    'hparams_seed': 0,
    'trial_seed': 0,
    'seed': 0,
    'hparams': f"'{json.dumps(hparams_irm_stronger)}'"
}

# Launch the experiment
run_experiment(irm_args_stronger)

🚀 Executing Command:
PYTHONPATH="/root/IbsATML/PA2/Domain Generalisation/code/domainbed" python -m domainbed.scripts.train --data_dir ./data/ --dataset PACS --algorithm IRM --test_env 3 --output_dir ./results/irm_stronger_penalty --hparams_seed 0 --trial_seed 0 --seed 0 --hparams '{"progress_bar": true, "irm_lambda": 1000.0, "irm_penalty_anneal_iters": 500}'


/venv/main/lib/python3.12/site-packages/outdated/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Environment:
	Python: 3.12.11
	PyTorch: 2.8.0+cu129
	Torchvision: 0.23.0+cu129
	CUDA: 12.9
	CUDNN: 91002
	NumPy: 2.1.2
	PIL: 11.0.0
Args:
	algorithm: IRM
	checkpoint_freq: None
	data_dir: ./data/
	dataset: PACS
	holdout_fraction: 0.2
	hparams: {"progress_bar": true, "irm_lambda": 1000.0, "irm_penalty_anneal_iters": 500}
	hparams_seed: 0
	output_dir: ./results/irm_stronger_penalty
	save_model_every_checkpoint: False
	seed: 0
	skip_model_save: False
	steps: None
	task: domain_generalization
	test_envs: [3]
	trial_seed: 0
	uda_holdout_fraction: 0
HParams:
	batch_size: 32
	class_balanced: False
	data_augmentation: True
	dinov2: False
	freeze_bn: False
	irm_lambda: 1000.0
	irm_penalty_anneal_iters: 500
	lars: False
	linear_steps: 500
	lr: 5e-05
	nonlinear_classifier: False
	progress_bar: True
	resnet18: False
	resnet50_augmix: True
	resnet_dropout: 0.0
	vit: False
	vit_attn_tune: False
	vit_dropout: 0.0
	weight_decay: 0.0
/venv/main/lib/python3.12/site-packages/torchvision/models/_utils.py: